In [1]:
#import libs i'll ensure them in middle of notebook also as per my requirements
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras
import nltk

2025-07-31 17:47:49.304552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 17:47:49.655764: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-31 17:47:49.803540: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-31 17:47:51.240726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
#loading csv
df=pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [3]:
df.shape

(28332, 24)

In [4]:
df.head(1)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."


In [5]:
#only 2 cols are imp as we have to perform normal sentiment analysis 
df["reviews.rating"].isna().sum()

0

In [6]:
df['reviews.text'].isna().sum()

0

In [7]:
df=df[['reviews.rating','reviews.text']]

In [8]:
df.isna().sum()

reviews.rating    0
reviews.text      0
dtype: int64

In [9]:
#dropping all the cols except these 2
df=df.dropna()

In [28]:
df.isna().sum()

reviews.rating    0
reviews.text      0
dtype: int64

In [25]:
text=df['reviews.text']

In [26]:
#now nlp part starts
import re
import string 

def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()  # remove leading/trailing whitespace
    return text


In [29]:
text=text.apply(clean_text)
text

0        i order  of them and one of the item is bad qu...
1        bulk is always the less expensive way to go fo...
2        well they are not duracell but for the price i...
3        seem to work as well as name brand batteries a...
4        these batteries are very long lasting the pric...
                               ...                        
28327    i got  of these for my  yr old twins my  yr ol...
28328    i bought this for my niece for a christmas gif...
28329    very nice for light internet browsing keeping ...
28330    this tablet does absolutely everything i want ...
28331    at ninety dollars the expectionations are low ...
Name: reviews.text, Length: 28332, dtype: object

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer(stop_words='english',max_features=5000)

In [31]:
x=vect.fit_transform(text)

In [32]:
def get_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Negative'

df['sentiment'] = df['reviews.rating'].apply(get_sentiment)

In [33]:
df['sentiment']=df['sentiment'].map({'Positive':2,'Neutral':1,'Negative':0})

In [34]:
y=df['sentiment']

In [35]:
from imblearn.over_sampling import SMOTE
sm=SMOTE()
x_new , y_new =sm.fit_resample(x,y)

In [39]:
x_new.shape[0]

76635

In [40]:
y_new.shape[0]

76635

In [36]:
df

,reviews.rating,reviews.text,sentiment
0,3,I order 3 of them and one of the item is bad q...,1
1,4,Bulk is always the less expensive way to go fo...,2
2,5,Well they are not Duracell but for the price i...,2
3,5,Seem to work as well as name brand batteries a...,2
4,5,These batteries are very long lasting the pric...,2
...,...,...,...
28327,5,I got 2 of these for my 8 yr old twins. My 11 ...,2
28328,4,I bought this for my niece for a Christmas gif...,2
28329,5,"Very nice for light internet browsing, keeping...",2
28330,5,This Tablet does absolutely everything I want!...,2


In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_new,y_new,test_size=0.2, random_state=42, stratify=y_new)

In [42]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000, class_weight='balanced')

In [43]:
model.fit(x_train,y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [44]:
y_pred=model.predict(x_test)

In [45]:
from sklearn.metrics import accuracy_score, classification_report
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.8903242643700658

In [46]:
print("\n",classification_report(y_test,y_pred))


               precision    recall  f1-score   support

           0       0.90      0.90      0.90      5109
           1       0.84      0.90      0.87      5109
           2       0.94      0.87      0.90      5109

    accuracy                           0.89     15327
   macro avg       0.89      0.89      0.89     15327
weighted avg       0.89      0.89      0.89     15327



In [47]:
df['sentiment'].value_counts()

sentiment
2    25545
0     1581
1     1206
Name: count, dtype: int64

In [48]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Example input
new_text = "This product is absolutely wonderful!"
clean_text = preprocess(new_text)

# Vectorize like training
vec_text = vect.transform([clean_text])

# Predict
prediction = model.predict(vec_text)
print(prediction)  # e.g., ['Positive']



[2]


In [49]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

Train Accuracy: 0.9037645984210869
Test Accuracy: 0.8903242643700658
